In [170]:
import datetime
import requests
import json
import pandas as pd
import concurrent.futures

In [171]:
username = 'API User'
token = 'API Token'
begin_time_set = datetime.datetime(2017, 7, 12, 0,0,0) 
refdes_streams = '/Users/knuth/Documents/ooi/repos/github/ooi_stats/test.csv'

In [172]:
# setup the base url for the request that will be built using the inputs above.
BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

# request only parameter 7 (time)
parameter = '7'

# read in csv (for now)
refdes_streams_df = pd.read_csv(refdes_streams)

In [174]:
# prepare time stamp manipulators and range of data requests
begin_time_str = begin_time_set.strftime('%Y-%m-%dT%H:%M:%S.000Z')
ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()
now = datetime.datetime.now()
days = abs(begin_time_set.date() - now.date())
days = int(days.days)
print days,"days of data since", begin_time_str, "will be requested for each refdes+stream."

8 days of data since 2017-07-12T00:00:00.000Z will be requested for each refdes+stream.


In [175]:
# iterate over reference designators, delivery methods and streams to build request url lists 
requests_dict = {}
for index, row in refdes_streams_df.iterrows():

    # start at the begin time set above
    begin_time = begin_time_set
    begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    
    #step forward by 1 day (86400 seconds) with each new request
    end_time = begin_time + datetime.timedelta(seconds=86400)
    end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')

    ref_des =  row['refdes']
    sub_site = ref_des[:8]
    platform = ref_des[9:14]
    instrument = ref_des[15:27]
    stream = row['stream']
    delivery_method = row['method']
    
    ref_des_list = []

    for i in range(days):
        
        
        
        request_url = '/'.join((BASE_URL, sub_site, platform, instrument, delivery_method, stream))
        request_url = request_url+'?beginDT='+begin_time_str+'&endDT='+end_time_str+'&limit=1000&parameters='+parameter
        
        ref_des_list.append(request_url)

        begin_time = begin_time + datetime.timedelta(seconds=86400)
        begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
        end_time = end_time + datetime.timedelta(seconds=86400)
        end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
        
    requests_dict[ref_des] = ref_des_list

    

In [176]:
# set up threads pool and execute requests
pool = concurrent.futures.ThreadPoolExecutor(max_workers=5)

In [177]:
# set up function to send requests
def request_data(url,username,token):
    auth = (username, token)
    return requests.get(url,auth=auth)

In [178]:
pool = concurrent.futures.ThreadPoolExecutor(max_workers=5)
for key, values in requests_dict.iteritems():
    print key
    future_to_url = {pool.submit(request_data, url, username, token): url for url in values}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        
        try:    
            data = future.result()
            data = data.json()
            timestamp = data[0]['time']
            timestamp = datetime.datetime.utcfromtimestamp(timestamp - ntp_delta).replace(microsecond=0)
            timestamp = timestamp.date()
            print timestamp
        except:
            data = future.result()
            data = data.json()
            print data['status']

RS03AXPS-SF03A-3A-FLORTD301
2017-07-12
2017-07-14
2017-07-15
2017-07-13
2017-07-18
2017-07-16
2017-07-17
2017-07-19
GA01SUMO-RII11-02-CTDMOQ012
2017-07-15
2017-07-12
2017-07-16
2017-07-14
2017-07-13
2017-07-17
2017-07-18
2017-07-19
CE05MOAS-GL319-04-DOSTAM000
{
  "message": "Query returned no results for primary stream", 
  "requestUUID": "f8be64d8-d4b5-4f54-a1be-7e8f74e4f48e"
}

{
  "message": "Query returned no results for primary stream", 
  "requestUUID": "96478985-ab8e-4733-b5cf-e8c72c6b31a0"
}

{
  "message": "Query returned no results for primary stream", 
  "requestUUID": "fffd921e-0c07-4d33-8f68-ac00548342d2"
}

{
  "message": "Query returned no results for primary stream", 
  "requestUUID": "d2f312f3-7c87-4bcb-ab9e-c412a59986a4"
}

{
  "message": "Query returned no results for primary stream", 
  "requestUUID": "3f0b9985-2656-4ffc-ab27-7eeece7ad05e"
}

{
  "message": "Query returned no results for primary stream", 
  "requestUUID": "c9db8dd3-6142-4caa-a2f7-b9136646846f"
}

{


SSLError: ("bad handshake: SysCallError(32, 'EPIPE')",)